# Comparison of spectrum between eNATL60 outputs and Altika Satellite for Region 1 and ASO months

Requisites :
 - notebook process-spectrum-comparison-eNATL60-region1-Altkia-ASO has successfully run and produced all the necessary result_*.nc in results_Altika-eNATL60-Region1-ASO
 - gonzag_cloud
 - climporn
 


In [1]:
import sys,os
from os import getenv
import warnings
warnings.filterwarnings("ignore")

In [2]:
GONZAG_DIR = '/home/jovyan/gonzag_cloud'
sys.path.append(GONZAG_DIR)
import gonzag as gz


In [3]:
import xarray as xr
import sys
import glob
import numpy as nmp
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker

from scipy.signal import detrend, tukey

%matplotlib inline


In [4]:
CLIMPORN_DIR = '/home/jovyan/climporn/python' ; # get it there: https://github.com/brodeau/climporn
sys.path.append(CLIMPORN_DIR)
import climporn as cp


### Params

For the model

In [5]:
model = 'eNATL60'
region = '1'
season = 'fma'
name_mod = model+'-Region'+region+'-'+season
name_ssh_mod = 'sossheig_bl'


For altimetry data

In [6]:
name_sat= 'Altika'
name_ssh_sat='sla_unfiltered'


### Data

In [7]:
fresults=sorted(glob.glob('../results/results_'+name_sat+'-'+name_mod+'/result_??.nc'))
dsn=xr.open_mfdataset(fresults,concat_dim='time',combine='nested')

In [8]:
#clean up some remaining Nans

ds=dsn.where(nmp.isnan(dsn[name_ssh_sat])==0, drop=True)

In [9]:
ds.time

<xarray.DataArray 'time' (time: 10001)>
array([1.391423e+18, 1.391423e+18, 1.391423e+18, ..., 1.396827e+18,
       1.396827e+18, 1.396827e+18])
Coordinates:
  * time     (time) float64 1.391e+18 1.391e+18 ... 1.397e+18 1.397e+18

In [9]:
print(ds[name_ssh_mod].values)

[-0.06541651  0.25607326  0.25922199 ... -0.21118741 -0.30652857
 -0.30651113]


### Spectrum

In [10]:
%%time

ISeg_beg, ISeg_end = gz.FindUnbrokenSegments( ds.time, ds.distance, ds[name_ssh_mod],\
                                             rcut_time=1.2e+09, rcut_dist=7.8 )
NbSeg, Nsl, IDEDSeg = gz.SegmentSelection(ISeg_beg, ISeg_end, np_valid_seg=70)



 *** [SegmentSelection()]: Will use 23 segments with a fixed length of 100 points!
     ==> 23 selected segments out of the 240 available (requested minimum length is 70 points)

CPU times: user 4min 12s, sys: 14.6 s, total: 4min 26s
Wall time: 4min 11s


In [17]:
IDEDSeg[15]

array([4865, 4964])

In [18]:
IDEDSeg2=nmp.delete(IDEDSeg, 15, axis=0)
NbSeg2=NbSeg-1

In [19]:
IDEDSeg2

array([[ 377,  476],
       [ 810,  909],
       [1147, 1246],
       [1388, 1487],
       [1547, 1646],
       [1746, 1845],
       [1918, 2017],
       [2091, 2190],
       [2413, 2512],
       [2833, 2932],
       [3540, 3639],
       [3753, 3852],
       [3925, 4024],
       [4098, 4197],
       [4263, 4362],
       [5616, 5715],
       [5959, 6058],
       [6281, 6380],
       [7605, 7704],
       [8087, 8186],
       [8672, 8771],
       [9454, 9553]])

In [20]:
XPs, XPm, rdist_sample = gz.Process4FFT( IDEDSeg2, ds.distance, ds[name_ssh_mod], ds[name_ssh_sat] )
Kwn, PwSpc_s, PwSpc_m = gz.ApplyFFT( IDEDSeg2, XPs, XPm, rdist_sample )

# Building our spectrum as the mean of the NbSeg spectra:
vps_mod = nmp.mean(PwSpc_m[:,:],axis=0)
vps_sat = nmp.mean(PwSpc_s[:,:],axis=0)


 *** [Process4FFT()]: applying linear detrending...
 *** [Process4FFT()]: centering about 0...
 *** [Process4FFT()]: applying "Tukey" tappering...

 *** [ApplyFFT()]: Applying FFT with a dx_sample of  6.603  km



In [21]:
# Blabla for the plot:
clr_sat = '#AD0000'
clr_mod = '#008ab8'

cinfrm = str(NbSeg)+' segments\n'+str(Nsl)+' points/segment\n'+r'$\Delta$d sat.: '+str(round(rdist_sample,1))+' km'

ii = cp.plot("pow_spectrum_ssh")(Kwn, vps_mod, clab1=name_mod+' ("'+name_ssh_mod+'")', clr1=clr_mod, lw1=5, \
                                 cinfo=cinfrm, logo_on=False, \
                                 L_min=13., L_max=500., P_min_y=-6, P_max_y=1, \
                                 l_show_k4=False, l_show_k5=True, l_show_k11o3=False, l_show_k2=True, \
                                 vk2=Kwn, vps2=vps_sat, clab2=name_sat+' ("'+name_ssh_sat+'")', clr2=clr_sat, lw2=4)

plt.savefig('../plots/spectrum_SSH_'+name_sat+'-'+name_mod+'.png')


findfont: Font family ['Trebuchet MS'] not found. Falling back to DejaVu Sans.
